<a href="https://colab.research.google.com/github/dawidstajszczyk/thesis_APO/blob/main/image_processing2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ekstrakcja danych ze strony: https://www.kaggle.com/datasets/dawidstajszczyk00/dice-r0ll

In [1]:
import zipfile
import os
from google.colab import drive
import shutil

# Wyczyść workspace
folder_path = '/content/extracted_images'
shutil.rmtree(folder_path, ignore_errors=True)

# Montuj Google Drive
drive.mount('/content/drive')

# Skopiuj plik kaggle.json do środowiska Colab
!cp '/content/drive/MyDrive/kaggle/kaggle.json' '/content'
os.environ['KAGGLE_CONFIG_DIR'] = "/content"

# Pobierz zestaw danych
!kaggle datasets download -d dawidstajszczyk00/dice-roll

# Ścieżka do katalogu, gdzie znajduje się pobrany plik ZIP
zip_file_path = '/content/dice-roll.zip'

# Katalog docelowy dla rozpakowanych danych
extracted_folder_path = '/content/extracted_images'

# Rozpakuj plik ZIP
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

Mounted at /content/drive
Dataset URL: https://www.kaggle.com/datasets/dawidstajszczyk00/dice-roll
License(s): Apache 2.0
100% 574M/577M [00:31<00:00, 23.8MB/s]
100% 577M/577M [00:31<00:00, 19.5MB/s]


Wyodrębnienie losowych próbek do zbioru testowego

In [52]:
from sklearn.model_selection import train_test_split
import os
import random

# Wyczyść workspace
folder_path = '/content/extracted_images/test'
shutil.rmtree(folder_path, ignore_errors=True)

# Ścieżka do foldera zawierającego klasy
base_path = '/content/extracted_images/'

# Lista klas
classes = ['1', '2', '3', '4', '5', '6']

# Lista zawierająca pełne ścieżki do folderów klas
class_paths = [os.path.join(base_path, class_name) for class_name in classes]

# Ścieżka do foldera testowego
test_path = os.path.join(extracted_folder_path, "test")

# Utwórz folder testowy, jeśli nie istnieje
os.makedirs(test_path, exist_ok=True)

# Zmienna przechowująca przewidywaną liczbę wyrzuconych oczek
predictions = []

# Podział każdego folderu klasy na zbiór treningowy i testowy
for i, class_path in enumerate(class_paths):
    images = os.listdir(class_path)

    _, test_images = train_test_split(images, test_size=0.02, random_state=42)

    for test_image in test_images:
        # dodaj ścieżkę oraz klasę zdjęcia do listy predictions
        predictions.append({'image': test_image, 'class': i+1})

        # Przenieś zdjęcie do folderu testowego
        src_path = os.path.join(class_path, test_image)
        dest_path = os.path.join(test_path, test_image)
        os.rename(src_path, dest_path)

# Po dodaniu obrazów do zbioru testowego, wymieszaj je
test_images = os.listdir(test_path)
random.shuffle(test_images)

Przetwarzanie obrazów - ciało funkcji

In [53]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

def count_black_dots(image_path):
    # Wczytaj obraz
    image = cv2.imread(image_path)

    # Przekształć obraz na odcienie szarości
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Zastosuj rozmycie medianowe w celu wygładzenia krawędzi
    blurred = cv2.medianBlur(gray, 25)

    # Ustaw parametry algorytmu HoughCircles
    minDist = 100 # Minimalny dystans pomiędzy wykrytymi okręgami
    param1 = 30
    param2 = 50
    minRadius = 5 # Minimalny promień okręgów
    maxRadius = 75 # Maksymalny promień okręgów

    # Wykryj okręgi za pomocą algorytmu HoughCircles
    circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, 1, minDist, param1=param1, param2=param2, minRadius=minRadius, maxRadius=maxRadius)



    # Przekonwertuj 'blurred' na format BGR, aby móc rysować kolorowe kontury
    blurred_bgr = cv2.cvtColor(blurred, cv2.COLOR_GRAY2BGR)

    # Narysuj znalezione okręgi na przetworzonym obrazie
    if circles is not None:
        # Zlicz ilość wykrytych okręgów
        black_dot_count = circles.shape[1]

        circles = np.uint16(np.around(circles))
        for i in circles[0,:]:
            cv2.circle(blurred_bgr, (i[0], i[1]), i[2], (110, 255, 100), 8)
    else:
      black_dot_count = 0
      print("Circles hasn't been detected.")

    return blurred_bgr, black_dot_count

In [54]:
from skimage import io, color
import shutil


# Wyczyść workspace
folder_path = '/content/extracted_images/processed_images'
shutil.rmtree(folder_path, ignore_errors=True)

# Ścieżka, w którym zostaną umieszczone przetworzone obrazy
processed_path = '/content/extracted_images/processed_images/'

# Utwórz folder processed_images, jeśli nie istnieje
os.makedirs(processed_path, exist_ok=True)

# Zmienna przechowująca nazwy zdjęć ze zbioru testowego
image_files = os.listdir(test_path)

for image_file in image_files:
    # Ścieżka do zdjęcia ze zbioru testowego
    image_path = os.path.join(test_path, image_file)

    if os.path.isfile(image_path):
        # Przetwórz obraz
        processed_image, dots_num = count_black_dots(image_path)

        # Utwórz ścieżkę dla obrazu przetworzonego
        processed_image_path = os.path.join(processed_path, image_file)

        # Zapisz przetworzony obraz w folderze processed_images
        io.imsave(processed_image_path, processed_image.astype(np.uint8), check_contrast=False)

        # Dodaj przewidywaną liczbę wyrzuconych oczek do listy predictions
        for item in predictions:
            if item['image'] == image_file:
                item['count'] = dots_num
                break
    else:
        print("Incorrect file.")

print("Images processed and saved to processed_images folder")

Images processed and saved to processed_images folder


Podsumowanie klasyfikacji

In [55]:
cnt = 0;

for result in predictions:
    print(f"Image {result['image']} was classified as {result['count']} and was of the class {result['class']}")
    if result['class'] == result['count']: cnt += 1

accuracy = cnt / len(predictions)
print("accuracy: ", accuracy)

Image IMG_9189.JPEG was classified as 1 and was of the class 1
Image IMG_7051.JPEG was classified as 1 and was of the class 1
Image IMG_9208.JPEG was classified as 1 and was of the class 1
Image IMG_9236.JPEG was classified as 2 and was of the class 2
Image IMG_9220.JPEG was classified as 2 and was of the class 2
Image IMG_9546.JPEG was classified as 2 and was of the class 2
Image IMG_9277.JPEG was classified as 3 and was of the class 3
Image IMG_6721.JPEG was classified as 3 and was of the class 3
Image IMG_9267.JPEG was classified as 3 and was of the class 3
Image IMG_9627.JPEG was classified as 4 and was of the class 4
Image IMG_6600.JPEG was classified as 4 and was of the class 4
Image IMG_9628.JPEG was classified as 4 and was of the class 4
Image IMG_9381.JPEG was classified as 5 and was of the class 5
Image IMG_6458.JPEG was classified as 5 and was of the class 5
Image IMG_9669.JPEG was classified as 6 and was of the class 5
Image IMG_9714.JPEG was classified as 6 and was of the 

Proces przetwarzania obrazów:
1. Ładowanie obrazu
2. Konwersja do skali szarości
3. Rozmycie Gaussowskie
4. Binaryzacja obrazu
5. Detekcja krawędzi
6. Dylatacja
7. Znalezenie konturów
7. Klasyfikacja konturów na podstawie obszaru i kolistości

Co dalej?
1. Stworzyć nowy dataset z różnorodnym tłem - 100 zdjęć na klasę
	- zrób te zdjęcia od nowa, raczej nie opłaca się przerabiać istniejących
2. Użyć 2-3% zbioru na klasyfikację wykorzystującą APO.
3. Użyć 100% zbioru do treningu i klasyfikacji wykorzystującą ML.
